# Базовые модели

- Naive baseline: завтра = сегодня
- EWMA baseline: скользящий стандарт

Используется для контроля адекватности продвинутых моделей.


In [2]:
import pandas as pd
import numpy as np
from pathlib import Path
import warnings
warnings.filterwarnings('ignore')

print("✅ Библиотеки загружены")


✅ Библиотеки загружены


## Базовые модели прогноза волатильности


In [3]:
def naive_forecast(returns, window=30):
    """
    Naive baseline: прогноз волатильности = текущая волатильность
    """
    current_vol = returns.rolling(window=window).std() * np.sqrt(252)
    forecast = current_vol.shift(1)  # Прогноз = предыдущее значение
    return forecast


def ewma_forecast(returns, span=30):
    """
    EWMA baseline: экспоненциально взвешенная волатильность
    """
    ewma_vol = returns.ewm(span=span).std() * np.sqrt(252)
    forecast = ewma_vol.shift(1)
    return forecast


def calculate_mae(actual, predicted):
    """Mean Absolute Error"""
    mask = ~(np.isnan(actual) | np.isnan(predicted))
    return np.mean(np.abs(actual[mask] - predicted[mask]))


def calculate_rmse(actual, predicted):
    """Root Mean Squared Error"""
    mask = ~(np.isnan(actual) | np.isnan(predicted))
    return np.sqrt(np.mean((actual[mask] - predicted[mask])**2))

print("✅ Функции базовых моделей загружены")


✅ Функции базовых моделей загружены


## Загрузка данных, создание прогнозов и сохранение данных


In [4]:
# === ОБРАБОТКА ВСЕХ ТИКЕРОВ ===
ML_ROOT = Path.cwd().parent  # Поднимаемся на уровень ML/
DATA_DIR = ML_ROOT / '02_feature_engineering' / 'data' / 'features'
OUTPUT_DIR = ML_ROOT / 'data' / 'models'
OUTPUT_DIR.mkdir(parents=True, exist_ok=True)

print(f"📂 Источник данных: {DATA_DIR}")
print(f"📂 Выход: {OUTPUT_DIR}")

# Получаем список всех тикеров
available_files = list(DATA_DIR.glob("*_with_targets.parquet"))
tickers = [f.stem.replace('_with_targets', '') for f in available_files]
print(f"📋 Найдено тикеров: {len(tickers)}\n")

# Сбор метрик по всем тикерам
all_metrics = []
target_col = 'target_vol_5d'
last_df = None
last_ticker = None

for ticker in tickers:
    try:
        df = pd.read_parquet(DATA_DIR / f"{ticker}_with_targets.parquet")
        
        # Создание прогнозов
        df['forecast_naive'] = naive_forecast(df['log_return'], window=30)
        df['forecast_ewma'] = ewma_forecast(df['log_return'], span=30)
        
        # Расчёт метрик
        mae_naive = calculate_mae(df[target_col].values, df['forecast_naive'].values)
        mae_ewma = calculate_mae(df[target_col].values, df['forecast_ewma'].values)
        rmse_naive = calculate_rmse(df[target_col].values, df['forecast_naive'].values)
        rmse_ewma = calculate_rmse(df[target_col].values, df['forecast_ewma'].values)
        
        # === СОХРАНЕНИЕ ДЛЯ КАЖДОГО ТИКЕРА ===
        # 1. Прогнозы
        df.to_parquet(OUTPUT_DIR / f"{ticker}_baseline_forecasts.parquet", index=False)
        
        # 2. Индивидуальные метрики
        ticker_metrics = pd.DataFrame({
            'model': ['Naive', 'EWMA'],
            'mae': [mae_naive, mae_ewma],
            'rmse': [rmse_naive, rmse_ewma]
        })
        ticker_metrics.to_csv(OUTPUT_DIR / f"{ticker}_baseline_metrics.csv", index=False)
        
        # Добавляем в сводную таблицу
        all_metrics.append({
            'ticker': ticker,
            'naive_mae': mae_naive,
            'naive_rmse': rmse_naive,
            'ewma_mae': mae_ewma,
            'ewma_rmse': rmse_ewma
        })
        
        # Сохраняем SBER для примера (или первый тикер)
        if ticker == 'SBER' or last_df is None:
            last_df = df.copy()
            last_ticker = ticker
        
        print(f"✅ {ticker}: MAE naive={mae_naive:.4f}, ewma={mae_ewma:.4f}")
        
    except Exception as e:
        print(f"❌ {ticker}: {e}")

# === СОХРАНЕНИЕ СВОДНОЙ ТАБЛИЦЫ ===
metrics_df = pd.DataFrame(all_metrics)
metrics_df.to_csv(OUTPUT_DIR / "all_baseline_metrics.csv", index=False)

print(f"\n{'='*60}")
print(f"📊 ИТОГОВЫЕ МЕТРИКИ ({len(all_metrics)} тикеров)")
print(f"{'='*60}")
print(metrics_df)

print(f"\n📈 Средние значения:")
print(metrics_df[['naive_mae', 'naive_rmse', 'ewma_mae', 'ewma_rmse']].mean())

print(f"\n{'='*60}")
print(f"📊 Пример прогнозов ({last_ticker}):")
print(f"{'='*60}")
print(last_df[last_df[target_col].notna()][['date', target_col, 'forecast_naive', 'forecast_ewma']].tail(10))

print(f"\n✅ Сохранено в {OUTPUT_DIR}:")
print(f"   • {len(tickers)} файлов *_baseline_forecasts.parquet")
print(f"   • {len(tickers)} файлов *_baseline_metrics.csv")
print(f"   • 1 файл all_baseline_metrics.csv")

📂 Источник данных: e:\Python\VolatilityChecker\MOEXScanner\ML\02_feature_engineering\data\features
📂 Выход: e:\Python\VolatilityChecker\MOEXScanner\ML\data\models
📋 Найдено тикеров: 31

✅ AFKS: MAE naive=0.1585, ewma=0.1540
✅ AFLT: MAE naive=0.1599, ewma=0.1559
✅ ALRS: MAE naive=0.1276, ewma=0.1247
✅ BELU: MAE naive=0.4127, ewma=0.4337
✅ BSPB: MAE naive=0.2024, ewma=0.2023
✅ CHMF: MAE naive=0.1536, ewma=0.1431
✅ FIVE: MAE naive=0.1552, ewma=0.1559
✅ GAZP: MAE naive=0.1791, ewma=0.1748
✅ GMKN: MAE naive=0.1204, ewma=0.1167
✅ HYDR: MAE naive=0.1268, ewma=0.1197
✅ IMOEX: MAE naive=0.1130, ewma=0.1117
✅ IRAO: MAE naive=0.1465, ewma=0.1413
✅ LENT: MAE naive=0.1782, ewma=0.1832
✅ LKOH: MAE naive=0.1270, ewma=0.1252
✅ MAGN: MAE naive=0.1351, ewma=0.1287
✅ MGNT: MAE naive=0.1376, ewma=0.1331
✅ MTSS: MAE naive=0.1561, ewma=0.1488
✅ NLMK: MAE naive=0.1304, ewma=0.1262
✅ NVTK: MAE naive=0.1458, ewma=0.1432
✅ OZON: MAE naive=0.2115, ewma=0.2025
✅ PIKK: MAE naive=0.1716, ewma=0.1645
✅ PLZL: MAE nai

## Сохранение результатов


In [8]:
OUTPUT_DIR = Path('data') / 'models'
OUTPUT_DIR.mkdir(parents=True, exist_ok=True)

# Сохранение результатов
output_path = OUTPUT_DIR / f"{ticker}_baseline_forecasts.parquet"
df.to_parquet(output_path, index=False)

# Сохранение метрик
metrics = pd.DataFrame({
    'model': ['Naive', 'EWMA'],
    'mae': [mae_naive, mae_ewma],
    'rmse': [rmse_naive, rmse_ewma]
})
metrics_path = OUTPUT_DIR / f"{ticker}_baseline_metrics.csv"
metrics.to_csv(metrics_path, index=False)

print(f"✅ Сохранено: {output_path}")
print(f"✅ Метрики: {metrics_path}")


✅ Сохранено: data\models\YNDX_baseline_forecasts.parquet
✅ Метрики: data\models\YNDX_baseline_metrics.csv
